In [1]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import Row
from pyspark.sql.types import *
from pyspark.sql import SQLContext
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import RandomForestClassifier

from PIL import Image
from PIL import ImageOps 
import numpy as np 
import os 
from IPython.display import display, HTML

import glob
import re

In [2]:
conf = SparkConf()
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '4G')
        .set('spark.driver.memory', '16G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)

# Resizing the images

In [11]:
def imageResize(basename,imageName):
    """
    resize image
    basename : eg. /home/username/XYZFolder
    image name : xyz.jpg
    New folder in the working directory will be created with '_resized' as suffix
    """
    new_width  = 128
    new_height = 128
    try:  
        img = Image.open(basename+"/"+imageName) # image extension *.png,*.jpg
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        img.save(basename+'_resized/'+imageName)
    except:
        os.mkdir(basename+'_resized/')
        img = Image.open(basename+"/"+imageName) # image extension *.png,*.jpg
        img = img.resize((new_width, new_height), Image.ANTIALIAS)
        img.save(basename+'_resized/'+imageName)

def resizer(folderPath):
    """
    to resize all files present in a folder
    resizer('/home/username/XYZFolder')
    """
    
    for subdir, dirs, files in os.walk(folderPath):
        for fileName in files:
#             try:
                #  print os.path.join(subdir, file)
                filepath = subdir + os.sep + fileName
                #  print filepath
                if filepath.endswith(".jpg" or ".jpeg" or ".png" or ".gif"):
                    imageResize(subdir,fileName)
#             except:
#                 print traceback.print_exc()

In [8]:
# resizer('wiki_crop/wiki_crop_new/')
# # went to wiki_crop/wiki_crop_new/_resized/

# Filtering & Converting images to pixels

In [12]:
path = 'wiki_crop/wiki_crop_new/_resized/'
def filterimage(path):
    my_sub_dir = glob.glob(path + '*.jpg')
    for i in my_sub_dir:
        if os.path.getsize(i) < 1000:
            # print(path + str(i) + '/' + str(j))
            os.remove(i)

#filter images that are corrupted
# filterimage(path)

In [15]:
def load_image2(infilename) :
    '''  
    convert image file to pixels
    load_image2('fileName')
    '''
    img = Image.open(infilename).convert('L')
    data = np.array(img)
    return data


def ageAtPhoto(fileName):
    '''
    get age at time of photo
    ageAtPhoto('full_path_to_file')
    10049200_1891-09-16_1958.jpg
    yob is 1891
    dtpt is 1958
    '''
    basename = fileName.split('/')[-1].split('_')
    birth = int(basename[1].split('-')[0])
    today = int(basename[2].split('.')[0])
    currAge = abs(today - birth)
    return currAge



def convertToNumpy(folder):
    '''
    get pixels and age for each image in a folder
    x_values, y_values = convertToNumpy(fileNames)    
    '''
    pixels = []
    ages = []
    filename =[]
    for fileName in folder:
        if fileName.endswith(".jpg" or ".jpeg" or ".png"):
            age = ageAtPhoto(fileName)
            if (age<100 and age>0):
                img_px = np.ravel(load_image2(fileName))
                pixels.append(img_px)
                ages.append(age)
                filename.append(fileName.split('/')[-1])
    return pixels, ages,filename

In [16]:
folderName = 'wiki_crop/wiki_crop_new/_resized/' 
fileNames = glob.glob(folderName +'*.jpg')
# only test 20 files for now
NumberOfFileToTrained =100000

x_values, y_values,filename = convertToNumpy(fileNames[:NumberOfFileToTrained])

# print x_values
# print y_values

In [17]:
len(y_values)

1975

# Putting the data on RDD and converting to DF

In [7]:
flat_rdd = sc.parallelize(x_values).map(lambda x : x.tolist()).map(lambda x: [int(element) for element in x])
# len(flat_rdd.take(5)[0])
age_rdd = sc.parallelize(y_values).map(lambda x:int(x))
# age_rdd.take(5)
f_name = sc.parallelize(filename)
combined = flat_rdd.zip(age_rdd).zip(f_name)
# combined.getNumPartitions()

In [14]:
combined.collect()

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


In [8]:
#Create a DataFrame
imageschema = StructType([
   StructField("features", ArrayType(elementType=IntegerType(),containsNull=False),True),
   StructField("label", IntegerType(),True),
   StructField("f_name", StringType(),True)
])
df = sqlContext.createDataFrame(combined.map(lambda x : Row(x[0][0][:],x[0][1],x[1])), imageschema)
df.show()

+--------------------+-----+--------------------+
|            features|label|              f_name|
+--------------------+-----+--------------------+
|[67, 67, 67, 65, ...|   21|31843216_1990-06-...|
|[139, 140, 142, 1...|   46|1000001952958_196...|
|[0, 0, 0, 0, 0, 0...|   98|12872267_1910-10-...|
|[224, 223, 216, 2...|   19|43481633_1995-11-...|
|[220, 219, 218, 2...|   52|1000001671401_195...|
|[3, 3, 3, 3, 3, 3...|   59|100000185210_1944...|
|[21, 19, 18, 21, ...|   27|1000007243240_197...|
|[196, 196, 196, 1...|   31|10000022821347_19...|
|[35, 35, 35, 35, ...|   61|10000029563647_19...|
|[23, 23, 23, 23, ...|   28|1000003156912_198...|
|[61, 60, 60, 62, ...|   39|10000026256606_19...|
|[83, 71, 57, 54, ...|   34|610477_1976-04-20...|
|[123, 123, 123, 1...|   45|30948829_1963-03-...|
|[204, 204, 204, 2...|   34|10000012277158_19...|
|[0, 0, 0, 0, 0, 0...|   27|1000001924882_194...|
|[252, 252, 252, 2...|   24|10000015230998_19...|
|[195, 200, 205, 2...|   63|10000028738868_19...|


In [9]:
df.printSchema()

root
 |-- features: array (nullable = true)
 |    |-- element: integer (containsNull = false)
 |-- label: integer (nullable = true)
 |-- f_name: string (nullable = true)



In [13]:
# Download df in json format
# df.write.format('json').save('project/dataset.json')

In [14]:
# Write df to MongoDB
df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").save()

# Read df from MongoDB
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()
df.show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[67.0,67.0,67.0,6...|   21|
|[139.0,140.0,142....|   46|
|[0.0,0.0,0.0,0.0,...|   98|
|[224.0,223.0,216....|   19|
|[220.0,219.0,218....|   52|
|[3.0,3.0,3.0,3.0,...|   59|
|[21.0,19.0,18.0,2...|   27|
|[196.0,196.0,196....|   31|
|[35.0,35.0,35.0,3...|   61|
|[23.0,23.0,23.0,2...|   28|
|[61.0,60.0,60.0,6...|   39|
|[83.0,71.0,57.0,5...|   34|
|[123.0,123.0,123....|   45|
|[204.0,204.0,204....|   34|
|[0.0,0.0,0.0,0.0,...|   27|
|[252.0,252.0,252....|   24|
|[195.0,200.0,205....|   63|
|[138.0,137.0,142....|   23|
|[103.0,103.0,102....|   28|
|[70.0,69.0,70.0,7...|   30|
+--------------------+-----+
only showing top 20 rows



In [16]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri","mongodb://127.0.0.1/msan697.images").load()

# Splitting the df to train/test

In [10]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf
list_to_vector_udf = udf(lambda l: Vectors.dense(l), VectorUDT())
df = df.select(list_to_vector_udf(df["features"]).alias("features"),'label')

dataset = df.randomSplit([0.8, 0.2])
train = dataset[0].cache()
test = dataset[1].cache()

In [ ]:
train.show()
test.show()

# Training the model

In [11]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(maxIter=10, fitIntercept=True)
lrmodel = lr.fit(train)


## Predictions

In [12]:
validpredict = lrmodel.transform(test)
validpredict.show()

+--------------------+-----+--------------------+--------------------+----------+
|            features|label|       rawPrediction|         probability|prediction|
+--------------------+-----+--------------------+--------------------+----------+
|[0.0,0.0,0.0,0.0,...|   22|[0.47553352010982...|[0.00151244331494...|      22.0|
|[0.0,0.0,0.0,0.0,...|   27|[0.47553352010982...|[0.00151244331494...|      22.0|
|[13.0,21.0,31.0,3...|   49|[0.86286628415266...|[6.31712801656277...|      34.0|
|[63.0,63.0,63.0,6...|   28|[2.25797846458182...|[1.22932906023510...|      29.0|
|[78.0,79.0,78.0,7...|   33|[1.63863691211937...|[1.03280974741870...|      31.0|
|[79.0,89.0,92.0,8...|   35|[1.58516471067865...|[1.94723527919132...|      26.0|
|[83.0,83.0,83.0,8...|   32|[1.84280598911004...|[7.18916993737185...|      31.0|
|[97.0,99.0,100.0,...|   33|[1.50851905888919...|[4.22414611451184...|      31.0|
|[103.0,103.0,102....|   28|[1.84717808615284...|[1.94383488453167...|      28.0|
|[196.0,196.0,19